### 1. 安装HuggingFace 并下载模型到本地

In [2]:
!pip install huggingface-hub -Uqq 
!pip install -U sagemaker

DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [2]:
!rm -rf ./LLM_qwen_int4_model

In [9]:
from huggingface_hub import snapshot_download
from pathlib import Path
local_model_path = Path("./LLM_qwen_14b_int4_model")


In [10]:
local_model_path.mkdir(exist_ok=True)
# model_name = "Qwen/Qwen-7B-Chat-Int4"
# commit_hash = "955bbfa8c8f0c592b7713f1186c2c9ab477ef862"


model_name = "Qwen/Qwen-14B-Chat-Int4"
commit_hash = "0f5e18f5f18b3ced68be965099f091189964ed85"

In [11]:
snapshot_download(repo_id=model_name, revision=commit_hash, cache_dir=local_model_path)

'LLM_qwen_14b_int4_model/models--Qwen--Qwen-14B-Chat-Int4/snapshots/0f5e18f5f18b3ced68be965099f091189964ed85'

### 2. 把模型拷贝到S3为后续部署做准备

In [12]:
import sagemaker
from sagemaker import image_uris
import boto3
import os
import time
import json

role = sagemaker.get_execution_role()  # execution role for the endpoint
sess = sagemaker.session.Session()  # sagemaker session for interacting with different AWS APIs
bucket = sess.default_bucket()  # bucket to house artifacts

region = sess._region_name
account_id = sess.account_id()

s3_client = boto3.client("s3")
sm_client = boto3.client("sagemaker")
smr_client = boto3.client("sagemaker-runtime")

In [13]:
s3_model_prefix = "LLM-RAG/workshop/LLM_qwen_14b_int4_model"  # folder where model checkpoint will go
model_snapshot_path = list(local_model_path.glob("**/snapshots/*"))[0]
s3_code_prefix = "LLM-RAG/workshop/LLM_qwen_14b_int4_deploy_code"
print(f"s3_code_prefix: {s3_code_prefix}")
print(f"model_snapshot_path: {model_snapshot_path}")

s3_code_prefix: LLM-RAG/workshop/LLM_qwen_14b_int4_deploy_code
model_snapshot_path: LLM_qwen_14b_int4_model/models--Qwen--Qwen-14B-Chat-Int4/snapshots/0f5e18f5f18b3ced68be965099f091189964ed85


In [14]:
!aws s3 cp --recursive {model_snapshot_path} s3://{bucket}/{s3_model_prefix}

upload: LLM_qwen_14b_int4_model/models--Qwen--Qwen-14B-Chat-Int4/snapshots/0f5e18f5f18b3ced68be965099f091189964ed85/.gitattributes to s3://sagemaker-us-east-2-946277762357/LLM-RAG/workshop/LLM_qwen_14b_int4_model/.gitattributes
upload: LLM_qwen_14b_int4_model/models--Qwen--Qwen-14B-Chat-Int4/snapshots/0f5e18f5f18b3ced68be965099f091189964ed85/NOTICE to s3://sagemaker-us-east-2-946277762357/LLM-RAG/workshop/LLM_qwen_14b_int4_model/NOTICE
upload: LLM_qwen_14b_int4_model/models--Qwen--Qwen-14B-Chat-Int4/snapshots/0f5e18f5f18b3ced68be965099f091189964ed85/LICENSE to s3://sagemaker-us-east-2-946277762357/LLM-RAG/workshop/LLM_qwen_14b_int4_model/LICENSE
upload: LLM_qwen_14b_int4_model/models--Qwen--Qwen-14B-Chat-Int4/snapshots/0f5e18f5f18b3ced68be965099f091189964ed85/README.md to s3://sagemaker-us-east-2-946277762357/LLM-RAG/workshop/LLM_qwen_14b_int4_model/README.md
upload: LLM_qwen_14b_int4_model/models--Qwen--Qwen-14B-Chat-Int4/snapshots/0f5e18f5f18b3ced68be965099f091189964ed85/assets/wecha

### 3. 模型部署准备（entrypoint脚本，容器镜像，服务配置）

In [15]:
#中国区需要替换为下面的image_uri
# inference_image_uri = (
#     f"727897471807.dkr.ecr.{region}.amazonaws.com.cn/djl-inference:0.23.0-deepspeed0.9.5-cu118"
# )


inference_image_uri = image_uris.retrieve(
    framework="djl-deepspeed",
    region=sess.boto_session.region_name,
    version="0.23.0"
)
print(f"Image going to be used is ---- > {inference_image_uri}")

Image going to be used is ---- > 763104351884.dkr.ecr.us-east-2.amazonaws.com/djl-inference:0.23.0-deepspeed0.9.5-cu118


In [16]:
!mkdir -p LLM_qwen_14b_int4_deploy_code

In [26]:
%%writefile LLM_qwen_14b_int4_deploy_code/model.py
from djl_python import Input, Output
import torch
import logging
import math
import os

from transformers import AutoTokenizer,AutoModelForCausalLM
from transformers.generation import GenerationConfig
# from auto_gptq import AutoGPTQForCausalLM


STOP_flag = "[DONE]"


def load_model(properties):
    tensor_parallel = properties["tensor_parallel_degree"]
    model_location = properties['model_dir']
    if "model_id" in properties:
        model_location = properties['model_id']
    logging.info(f"Loading model in {model_location}")
    
    tokenizer = AutoTokenizer.from_pretrained(model_location, trust_remote_code=True)
    model = AutoModelForCausalLM.from_pretrained(model_location, device_map="auto", trust_remote_code=True).eval()
    model.generation_config = GenerationConfig.from_pretrained(model_location, trust_remote_code=True) # 可指定不同的生成长度、top_p等相关超参

    return model, tokenizer, model.generation_config


model = None
tokenizer = None
generator = None
config = None

def stream_items(prompt, history, max_length, top_p, temperature):
    global model, tokenizer, config
    size = 0
    response = ""
    config.max_new_tokens = max_length
    config.top_p = top_p
    
    ##传入temperature会报错
    ##model.generation_config.temperature = temperature 
    res_generator = model.chat_stream(tokenizer, prompt, history=history,generation_config=config)
    for response in res_generator:
        this_response = response[size:]
        size = len(response)
        stream_buffer = { "outputs":this_response,"finished": False}
        yield stream_buffer
    ## stop
    # yield {"query": prompt, "outputs": STOP_flag, "response": response, "history": [], "finished": True}


def handle(inputs: Input):
    global model, tokenizer,config
    if not model:
        model, tokenizer,config = load_model(inputs.get_properties())

    if inputs.is_empty():
        return None
    data = inputs.get_as_json()
    
    input_sentences = data["inputs"]
    params = data["parameters"]
    history = data.get("history",[])
    stream = data.get('stream')
    print(f'input prompt:{input_sentences}')   
    print(f'generation config:{config}')
    outputs = Output()
    if stream:
        outputs.add_property("content-type", "application/jsonlines")
        outputs.add_stream_content(stream_items(input_sentences,history=history,**params))
    else:
        config.max_new_tokens = params.get('max_length',1024)
        config.top_p = params.get('top_p',1)
        response, history = model.chat(tokenizer, input_sentences, history=history,generation_config=config)
        result = {"outputs": response, "history" : history}
        outputs.add_as_json(result)
        
    return outputs

Overwriting LLM_qwen_14b_int4_deploy_code/model.py


#### Note: option.s3url 需要按照自己的账号进行修改

In [27]:
%%writefile LLM_qwen_14b_int4_deploy_code/serving.properties
engine=Python
option.tensor_parallel_degree=1
option.enable_streaming=True
option.predict_timeout=240
option.s3url = s3://sagemaker-us-east-2-946277762357/LLM-RAG/workshop/LLM_qwen_14b_int4_model/

Overwriting LLM_qwen_14b_int4_deploy_code/serving.properties


#### 注意: 必须把transformers升级到4.27.1以上，否则会出现 [Issue344](https://github.com/THUDM/ChatGLM-6B/issues/344)

如果是中国区建议添加国内的pip镜像,如下代码所示
```
%%writefile LLM_chatglm_deploy_code/requirements.txt
-i https://pypi.tuna.tsinghua.edu.cn/simple
transformers==4.28.1
```

In [28]:
%%writefile LLM_qwen_14b_int4_deploy_code/requirements.txt
transformers==4.32.0
accelerate
tiktoken
einops
scipy
transformers_stream_generator==0.0.4
peft
deepspeed
auto-gptq
optimum

Overwriting LLM_qwen_14b_int4_deploy_code/requirements.txt


In [29]:
# !pip install auto-gptq

In [30]:
!rm model.tar.gz
!cd LLM_qwen_14b_int4_deploy_code && rm -rf ".ipynb_checkpoints"
!tar czvf model.tar.gz LLM_qwen_14b_int4_deploy_code

LLM_qwen_14b_int4_deploy_code/
LLM_qwen_14b_int4_deploy_code/model.py
LLM_qwen_14b_int4_deploy_code/requirements.txt
LLM_qwen_14b_int4_deploy_code/serving.properties


In [31]:
s3_code_artifact = sess.upload_data("model.tar.gz", bucket, s3_code_prefix)
print(f"S3 Code or Model tar ball uploaded to --- > {s3_code_artifact}")

S3 Code or Model tar ball uploaded to --- > s3://sagemaker-us-east-2-946277762357/LLM-RAG/workshop/LLM_qwen_14b_int4_deploy_code/model.tar.gz


### 4. 创建模型 & 创建endpoint

In [32]:
from sagemaker.utils import name_from_base
import boto3

model_name = name_from_base(f"qwen-14B-int4") #Note: Need to specify model_name
print(model_name)
print(f"Image going to be used is ---- > {inference_image_uri}")

create_model_response = sm_client.create_model(
    ModelName=model_name,
    ExecutionRoleArn=role,
    PrimaryContainer={
        "Image": inference_image_uri,
        "ModelDataUrl": s3_code_artifact
    },
    
)
model_arn = create_model_response["ModelArn"]

print(f"Created Model: {model_arn}")

qwen-14B-int4-2023-09-26-01-30-53-032
Image going to be used is ---- > 763104351884.dkr.ecr.us-east-2.amazonaws.com/djl-inference:0.23.0-deepspeed0.9.5-cu118
Created Model: arn:aws:sagemaker:us-east-2:946277762357:model/qwen-14b-int4-2023-09-26-01-30-53-032


In [33]:
endpoint_config_name = f"{model_name}-config"
endpoint_name = f"{model_name}-endpoint"

#Note: ml.g4dn.2xlarge 也可以选择
endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "VariantName": "variant1",
            "ModelName": model_name,
            "InstanceType": "ml.g5.2xlarge",
            "InitialInstanceCount": 1,
            # "VolumeSizeInGB" : 400,
            # "ModelDataDownloadTimeoutInSeconds": 2400,
            "ContainerStartupHealthCheckTimeoutInSeconds": 10*60,
        },
    ],
)
endpoint_config_response

{'EndpointConfigArn': 'arn:aws:sagemaker:us-east-2:946277762357:endpoint-config/qwen-14b-int4-2023-09-26-01-30-53-032-config',
 'ResponseMetadata': {'RequestId': '1d97e0c2-ae8b-4e17-ac4b-b0f2166d16f3',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '1d97e0c2-ae8b-4e17-ac4b-b0f2166d16f3',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '125',
   'date': 'Tue, 26 Sep 2023 01:30:54 GMT'},
  'RetryAttempts': 0}}

In [34]:
create_endpoint_response = sm_client.create_endpoint(
    EndpointName=f"{endpoint_name}", EndpointConfigName=endpoint_config_name
)
print(f"Created Endpoint: {create_endpoint_response['EndpointArn']}")

Created Endpoint: arn:aws:sagemaker:us-east-2:946277762357:endpoint/qwen-14b-int4-2023-09-26-01-30-53-032-endpoint


#### 持续检测模型部署进度

In [35]:
import time
resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = resp["EndpointStatus"]
print("Status: " + status)

while status == "Creating":
    time.sleep(60)
    resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
    status = resp["EndpointStatus"]
    print("Status: " + status)

print("Arn: " + resp["EndpointArn"])
print("Status: " + status)

Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: InService
Arn: arn:aws:sagemaker:us-east-2:946277762357:endpoint/qwen-14b-int4-2023-09-26-01-30-53-032-endpoint
Status: InService


### 5. 模型测试

In [46]:
%%time
import json
import boto3

smr_client = boto3.client("sagemaker-runtime")

parameters = {
  "max_length": 1024,
  "temperature": 0.1,
  "top_p":0.8
}

CPU times: user 7.03 ms, sys: 1.19 ms, total: 8.22 ms
Wall time: 15.6 ms


In [47]:
import io


class StreamScanner:
    """
    A helper class for parsing the InvokeEndpointWithResponseStream event stream. 
    
    The output of the model will be in the following format:
    ```
    b'{"outputs": [" a"]}\n'
    b'{"outputs": [" challenging"]}\n'
    b'{"outputs": [" problem"]}\n'
    ...
    ```
    
    While usually each PayloadPart event from the event stream will contain a byte array 
    with a full json, this is not guaranteed and some of the json objects may be split across
    PayloadPart events. For example:
    ```
    {'PayloadPart': {'Bytes': b'{"outputs": '}}
    {'PayloadPart': {'Bytes': b'[" problem"]}\n'}}
    ```
    
    This class accounts for this by concatenating bytes written via the 'write' function
    and then exposing a method which will return lines (ending with a '\n' character) within
    the buffer via the 'readlines' function. It maintains the position of the last read 
    position to ensure that previous bytes are not exposed again. 
    """
    
    def __init__(self):
        self.buff = io.BytesIO()
        self.read_pos = 0
        
    def write(self, content):
        self.buff.seek(0, io.SEEK_END)
        self.buff.write(content)
        
    def readlines(self):
        self.buff.seek(self.read_pos)
        for line in self.buff.readlines():
            if line[-1] != b'\n':
                self.read_pos += len(line)
                yield line[:-1]
                
    def reset(self):
        self.read_pos = 0

## Stream

In [59]:
# prompts1 = """你"""
import time

start = time.time()
prompts1 = """写一篇500字的科幻小说，背景关于宇宙战争"""
# prompts1 = """AWS Clean Rooms 的FAQ文档有提到 Q: 是否发起者和数据贡献者都会被收费？A: 是单方收费，只有查询的接收方会收费。
# 请问AWS Clean Rooms是多方都会收费吗？
# """
prompts1 = "请根据以下三个反引号中的内容，回答用户的问题，如果三个反引号中的内容的包含markdown格式的内容，如参考图片，链接等，请尽可能利用并按markdown格式输出。如果三个反引号中的内容为空，则回答不知道，不要随意发挥和编造答案。 \n``` \n\nSageMaker 分布式模型并行库概述\nSageMaker 包括分布式模型并行库，有助于在多个计算节点上有效地分发和训练 DL 模型，克服了在单个 GPU 上训练模型的相关限制。此外，该库还允许您利用 EFA 支持的设备获得最佳分布式训练，从而通过低延迟、高吞吐量和操作系统旁路来提高节点间通信性能。\nSageMaker 分布式模型并行库概述\n由于像 GPT-J 这样的大型模型具有数十亿个参数，其 GPU 内存占用量超过了单个芯片，因此必须在多个 GPU 之间对这种模型进行分区。SageMaker 模型并行（SMP）库支持在多个 GPU 之间自动对模型进行分区。使用 SageMaker 模型并行功能，SageMaker 会代表您运行初始分析作业，以分析模型的计算和内存需求。然后，这些信息用于决定如何在 GPU 之间对模型进行分区，以最大限度地提高目标，如最大限度地提高速度或最大限度地减少内存占用量。\nSageMaker 分布式模型并行库概述\n这项功能还支持可选的管道运行调度，以最大限度地提高可用 GPU 的总体利用率。前向传递期间的激活传播和后向传递期间的梯度传播需要顺序计算，这限制了 GPU 的使用量。SageMaker 通过将小批次拆分为微批次，以便在不同的 GPU 上并行处理，从而利用管道运行调度克服了顺序计算限制。SageMaker 模型并行支持两种管道运行模式：\n\n简单管道 – 该模式下，每个微批次的前向传递结束后，才开始后向传递。\n交错管道 – 该模式下，微批次的后向运行尽可能优先。这样可以更快地释放用于激活的内存，从而更有效地利用内存。\n\nSageMaker 分布式模型并行库概述 - 使用 SageMaker 模型并行库\n请参阅以下代码：\n\n@smp.step\ndef train_step(model, data, target):\n    output = model(data)\n    loss = F.nll_loss(output, target, reduction=\"mean\")\n    model.backward(Loss)\n    \n    return output, loss\n\nwith smp.tensor_parallelism():\n    model = AutoModelForCausalLM.from_config(model_config)\n    \nmodel = smp.DistributedModel (model)\noptimizer = smp.DistributedOptimizer(optimizer)Python\nSageMaker 分布式模型并行库概述 - 使用 SageMaker 模型并行库\nSageMaker 模型并行库的张量并行为以下 Hugging Face 变换器模型提供了[开箱即用的支持](https://docs.aws.amazon.com/sagemaker/latest/dg/model-parallel-extended-features-pytorch-hugging-face.html)：\n\n[GPT-2](https://github.com/aws/amazon-sagemaker-examples/tree/main/training/distributed_training/pytorch/model_parallel/gpt2)、[BERT](https://github.com/aws/amazon-sagemaker-examples/tree/main/training/distributed_training/pytorch/model_parallel/bert) 和 RoBERTa（在 SMP 库 v1.7.0 及更高版本中可用）\n[GPT-J](https://github.com/aws/amazon-sagemaker-examples/tree/main/training/distributed_training/pytorch/model_parallel/gpt-j)（在 SMP 库 v1.8.0 及更高版本中可用）\nGPT-Neo（在 SMP 库 v1.10.0 及更高版本中可用）\nSageMaker 分布式模型并行库概述 - 使用 SageMaker 模型并行库\n使用 SMP 库进行性能调整的最佳实践\n在训练大型模型时，请考虑以下步骤，以便模型能以合理的批次大小在 GPU 内存中运行：\n\n为了提高性能，建议使用 GPU 内存更高、带宽互连更高的实例，如 p4d 和 p4de 实例。\n在大多数情况下，可以启用[优化器状态分片](https://docs.aws.amazon.com/sagemaker/latest/dg/model-parallel-extended-features-pytorch-optimizer-state-sharding.html)，当模型有多个副本（已启用数据并行性）时，优化器状态分片会很有用。可以通过在 modelparallel 配置中设置 \"shard_optimizer_state\": True 来开启优化器状态分片。\n使用[激活检查点](https://docs.aws.amazon.com/sagemaker/latest/dg/model-parallel-extended-features-pytorch-activation-checkpointing.html)，这是一种通过清除某些层的激活并在模型中选定模块的后向传递期间重新计算来减少内存使用量的技术。\n使用[激活卸载](https://docs.aws.amazon.com/sagemaker/latest/dg/model-parallel-extended-features-pytorch-activation-offloading.html)，这是一项可以进一步减少内存使用量的附加功能。要使用激活卸载，请在 modelparallel 配置中设置 \"offload_activations\": True。当激活检查点和管道并行功能处于开启状态且微批次数大于一时使用。\n启用[张量并行](https://docs.aws.amazon.com/sagemaker/latest/dg/model-parallel-extended-features-pytorch-tensor-parallelism.html)并增加并行度，其中并行度是 2 的幂。通常，出于性能考虑，张量并行仅限于一个节点内。\n\nSageMaker 分布式模型并行库概述 - 张量并行技术\n各个层或 nn.Modules 利用张量并行技术划分到各个设备上，以便同时运行。下图是一个最简单的例子，说明该库如何将一个模型分为四层，以实现双向张量并行性（\"tensor_parallel_degree\": 2）。每个模型副本的层被一分为二（分成两半），并分布在两个 GPU 之间。本例中的数据并行度为八，因为模型并行配置还包括 \"pipeline_parallel_degree\": 1 和 \"ddp\": True。该库管理张量分布式模型各副本之间的通信。\nSageMaker 分布式模型并行库概述 - 张量并行技术\n![参考图片:张量并行技术](https://d2908q01vomqb2.cloudfront.net/f1f836cb4ea6efb2a0b1b99f41ad8b103eff4b59/2023/06/05/ML-8585-tensorp.jpg)\nSageMaker 分布式模型并行库概述 - 张量并行技术\n这一功能的好处是，您可以选择要将张量并行应用于哪些层或哪些层的子集。要深入了解 PyTorch 的张量并行性和其他节省内存的功能，以及如何设置管道和张量并行性的组合，请参阅 [PyTorch 的 SageMaker 模型并行库的扩展功能](https://docs.aws.amazon.com/sagemaker/latest/dg/model-parallel-extended-features-pytorch.html)。\n```\n问:Sagemaker如何使用张量并行技术对模型分层"
response_model = smr_client.invoke_endpoint_with_response_stream(
            EndpointName=endpoint_name,
            Body=json.dumps(
            {
                "inputs": prompts1,
                "parameters": parameters,
                "history" : [],
                "stream":True
            }
            ),
            ContentType="application/json",
        )

event_stream = response_model['Body']
scanner = StreamScanner()
for event in event_stream:
    scanner.write(event['PayloadPart']['Bytes'])
    for line in scanner.readlines():
        try:
            resp = json.loads(line)
            # print(resp)
            print(resp.get("outputs")['outputs'], end='')
        except Exception as e:
            print(line)
            continue
print (f"\ntime:{time.time()-start} s")

SageMaker支持张量并行技术，它可以将一个模型分为多个层，并将这些层分布在多个设备上同时运行。这可以帮助我们利用多个GPU进行分布式训练，并最大限度地提高训练速度和减少内存使用量。在使用张量并行技术时，我们可以选择要将张量并行应用于哪些层或哪些层的子集，以适应不同的需求。
time:5.816547393798828 s


## None stream

In [58]:

endpoint_name= 'qwen-14B-int4-2023-09-26-01-30-53-032-endpoint'
prompts1 = """AWS Clean Rooms 的FAQ文档有提到 Q: 是否发起者和数据贡献者都会被收费？A: 是单方收费，只有查询的接收方会收费。
请问AWS Clean Rooms是多方都会收费吗？
"""
prompts1 = """写一篇500字的科幻小说，背景关于宇宙战争"""
start = time.time()
response_model = smr_client.invoke_endpoint(
            EndpointName=endpoint_name,
            Body=json.dumps(
            {
                "inputs": prompts1,
                "parameters": parameters,
                "history" : [],
            }
            ),
            ContentType="application/json",
        )

resp = response_model['Body'].read()
print (f"\ntime:{time.time()-start} s")
# print(resp.decode('utf8'))
print(json.loads(resp)['outputs'])



time:13.683655500411987 s
这是一个未来的世界，人类文明已经拓展到整个宇宙。但是，随着人类的探索，他们发现了一个新的敌人——来自外太空的侵略者。

这些外星人拥有强大的科技和军队，他们的目标是消灭人类文明，占领整个宇宙。人类的军队与他们进行了一场激烈的战斗，但是，他们的科技太过先进，人类的军队很快就败下阵来。

在这个关键时刻，一个名叫李文的科学家提出了一个计划。他发现，这些外星人的科技是基于一个叫做“黑盒”的设备，只要摧毁这个黑盒，他们的科技就会失效。

于是，李文组织了一支勇敢的队伍，他们决定去外太空寻找这个黑盒，并将其摧毁。他们乘坐着一艘先进的宇宙飞船，穿越了宇宙中的重重难关，最终找到了这个黑盒。

在一场激烈的战斗中，他们成功地摧毁了这个黑盒，外星人的科技也因此失效。人类军队乘机反击，最终成功地打败了外星人，保护了人类的文明。

从此以后，人类开始更加珍惜他们的科技和文明，也更加尊重和保护宇宙中的其他生物。


#### 清除模型Endpoint和config

In [111]:
!aws sagemaker delete-endpoint --endpoint-name {endpoint_name}

In [112]:
!aws sagemaker delete-endpoint-config --endpoint-config-name {endpoint_config_name}

In [113]:
!aws sagemaker delete-model --model-name {model_name}